In [1]:
# Risk analysis of mag7 stocks
import yfinance as yf

risk_free_rate=0.02

stocks=['AAPL','NVDA','MSFT','GOOG','GOOGL','AMZN','META','TSLA']
data=[yf.Ticker(i).history(period="max") for i in stocks]
market_caps=[yf.Ticker(i).info['marketCap'] for i in stocks]
spx=yf.Ticker('^GSPC').history(period="max")



In [2]:
dates=[i.index[0] for i in data]
cutoff_date=max(dates)

spx=spx[spx.index>=cutoff_date]
spx['daily_return']=spx['Close'].pct_change()
spx.dropna(inplace=True)
for i in range(len(stocks)):
    data[i]=data[i][data[i].index>=cutoff_date]
    data[i]['daily_return']=data[i]['Close'].pct_change()
    data[i].dropna(inplace=True)
    


In [11]:
import numpy as np
import pandas as pd

monthly_returns=np.zeros(len(stocks))
stdevs=np.zeros(len(stocks))
betas=np.zeros(len(stocks))
treynor_ratios=np.zeros(len(stocks))
value_at_risk95=np.zeros(len(stocks))
value_at_risk99=np.zeros(len(stocks))

for i in range(len(stocks)):
    monthly_returns[i]=np.power(np.mean(data[i]['daily_return'])+1,21)-1
    stdevs[i]=np.std(data[i]['daily_return'])*np.sqrt(21)
    betas[i] = np.cov(data[i]['daily_return'],spx['daily_return'])[0,1]/np.var(spx['daily_return'])
    treynor_ratios[i]=(monthly_returns[i]-risk_free_rate)/betas[i]
    value_at_risk95[i]=monthly_returns[i]-2*stdevs[i]
    value_at_risk99[i]=monthly_returns[i]-3*stdevs[i]

df=pd.DataFrame({'Stock':stocks,'Monthly Return %':monthly_returns*100,'Standard Deviation %':stdevs*100,'Beta':betas,'Treynor%':treynor_ratios*100,'Value at Risk 95%':value_at_risk95*100,'Value at Risk 99%':value_at_risk99*100})

In [14]:
# Calculate market cap weights
weights = np.array(market_caps) / sum(market_caps)

# Calculate portfolio metrics
port_monthly_return = np.sum(monthly_returns * weights) * 100
port_std = np.sqrt(np.sum(weights**2 * stdevs**2)) * 100
port_beta = np.sum(betas * weights)
port_treynor = ((port_monthly_return/100 - risk_free_rate) / port_beta) * 100
port_var95 = (port_monthly_return/100 - 2 * port_std/100) * 100
port_var99 = (port_monthly_return/100 - 3 * port_std/100) * 100

# Add portfolio row to DataFrame
portfolio_row = pd.DataFrame({
    'Stock': ['Portfolio'],
    'Monthly Return %': [port_monthly_return],
    'Standard Deviation %': [port_std],
    'Beta': [port_beta],
    'Treynor%': [port_treynor],
    'Value at Risk 95%': [port_var95],
    'Value at Risk 99%': [port_var99]
})

df = pd.concat([df, portfolio_row], ignore_index=True)

# Add S&P 500 metrics
spx_monthly_return = (np.power(np.mean(spx['daily_return'])+1, 21)-1) * 100
spx_std = np.std(spx['daily_return']) * np.sqrt(21) * 100
spx_beta = 1.0  # By definition
spx_treynor = (spx_monthly_return/100 - risk_free_rate) * 100
spx_var95 = (spx_monthly_return/100 - 2 * spx_std/100) * 100
spx_var99 = (spx_monthly_return/100 - 3 * spx_std/100) * 100

spx_row = pd.DataFrame({
    'Stock': ['S&P 500'],
    'Monthly Return %': [spx_monthly_return],
    'Standard Deviation %': [spx_std],
    'Beta': [spx_beta],
    'Treynor%': [spx_treynor],
    'Value at Risk 95%': [spx_var95],
    'Value at Risk 99%': [spx_var99]
})

df = pd.concat([df, spx_row], ignore_index=True)
df

,Stock,Monthly Return %,Standard Deviation %,Beta,Treynor%,Value at Risk 95%,Value at Risk 99%
0,AAPL,2.145608,8.078794,1.160431,0.125478,-14.011979,-22.090772
1,NVDA,5.068713,12.966157,1.680538,1.826030,-20.863602,-33.829759
2,MSFT,2.220046,7.537822,1.193041,0.184442,-12.855598,-20.393420
3,GOOG,2.032156,7.850489,1.136128,0.028304,-13.668821,-21.519310
4,GOOGL,2.024377,7.844376,1.138127,0.021418,-13.664375,-21.508750
5,AMZN,2.459792,9.242113,1.166371,0.394207,-16.024433,-25.266546
6,META,2.533423,11.502696,1.244581,0.428597,-20.471968,-31.974664
7,TSLA,5.015559,16.484854,1.496527,2.015039,-27.954148,-44.439002
8,Portfolio,2.872280,3.621346,1.276612,0.683278,-4.370412,-7.991758
9,Portfolio,2.872280,3.621346,1.276612,0.683278,-4.370412,-7.991758
